1. Взять данные.
2. Загрузить в PySpark.
3. При помощи VectorAssembler преобразовать все колонки с признаками в одну (использовать Pipeline — опционально).
4. Разбить данные на train и test.
5. Создать модель логистической регреcсии или модель дерева и обучить её.
6. Воспользоваться MulticlassClassificationEvaluator для оценки качества на train и test множестве.

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier


In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

24/04/26 17:09:40 WARN Utils: Your hostname, MacBook-Pro-Vadim.local resolves to a loopback address: 127.0.0.1; using 192.168.139.65 instead (on interface en0)
24/04/26 17:09:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/26 17:09:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/26 17:09:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read.csv('iris.csv', header=True, inferSchema=True)

In [4]:
df.show()

+------------+-----------+------------+-----------+-------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|
+------------+-----------+------------+-----------+-------+-----------+
|         5.1|        3.5|         1.4|        0.2| Setosa|          0|
|         4.9|        3.0|         1.4|        0.2| Setosa|          0|
|         4.7|        3.2|         1.3|        0.2| Setosa|          0|
|         4.6|        3.1|         1.5|        0.2| Setosa|          0|
|         5.0|        3.6|         1.4|        0.2| Setosa|          0|
|         5.4|        3.9|         1.7|        0.4| Setosa|          0|
|         4.6|        3.4|         1.4|        0.3| Setosa|          0|
|         5.0|        3.4|         1.5|        0.2| Setosa|          0|
|         4.4|        2.9|         1.4|        0.2| Setosa|          0|
|         4.9|        3.1|         1.5|        0.1| Setosa|          0|
|         5.4|        3.7|         1.5|        0.2| Setosa|     

In [10]:
df.describe().show()

+-------+------------------+-------------------+------------------+------------------+---------+------------------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  variety|       variety_num|
+-------+------------------+-------------------+------------------+------------------+---------+------------------+
|  count|               150|                150|               150|               150|      150|               150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|     NULL|               1.0|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|     NULL|0.8192319205190406|
|    min|               4.3|                2.0|               1.0|               0.1|   Setosa|                 0|
|    max|               7.9|                4.4|               6.9|               2.5|Virginica|                 2|
+-------+------------------+-------------------+------------------+-----

In [13]:
pipeline = Pipeline(stages=
[
    VectorAssembler(
        inputCols = ['sepal_length','sepal_width', 'petal_length', 'petal_width'],
        outputCol='features')
])

In [15]:
pipeline_fit = pipeline.fit(df)
df_feat = pipeline_fit.transform(df)

In [16]:
df_feat.show()

+------------+-----------+------------+-----------+-------+-----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|         features|
+------------+-----------+------------+-----------+-------+-----------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|          0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|          0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|          0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|          0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|          0|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|          0|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| Setosa|          0|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2| Setosa|          0|[5.0,3.4,1.5,0.2]|
|         

In [17]:
train, test = df_feat.randomSplit([0.7, 0.3])

In [21]:
lr = LogisticRegression(featuresCol='features', labelCol='variety_num')
lr = lr.fit(train)

In [22]:
train_pred = lr.transform(train)
test_pred = lr.transform(test)

In [23]:
train_pred.show()

+------------+-----------+------------+-----------+----------+-----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|variety_num|         features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+-----------+-----------------+--------------------+--------------------+----------+
|         4.4|        2.9|         1.4|        0.2|    Setosa|          0|[4.4,2.9,1.4,0.2]|[674.364309024481...|[1.0,2.1717266695...|       0.0|
|         4.4|        3.0|         1.3|        0.2|    Setosa|          0|[4.4,3.0,1.3,0.2]|[712.669171340770...|[1.0,8.1174444199...|       0.0|
|         4.6|        3.1|         1.5|        0.2|    Setosa|          0|[4.6,3.1,1.5,0.2]|[688.843358170942...|[1.0,4.3979956715...|       0.0|
|         4.6|        3.2|         1.4|        0.2|    Setosa|          0|[4.6,3.2,1.4,0.2]|[727.148220487231...|[1.0,1.6438

In [28]:
ev = MulticlassClassificationEvaluator(labelCol='variety_num')

In [30]:
ev.evaluate(train_pred), ev.evaluate(test_pred)

(1.0, 0.959086491739553)

In [35]:
tree = DecisionTreeClassifier(labelCol='variety_num')
tree = tree.fit(train)

In [38]:
tr_train_pred = lr.transform(train)
tr_test_pred = lr.transform(test)

In [40]:
ev.evaluate(tr_train_pred), ev.evaluate(tr_test_pred)

(1.0, 0.959086491739553)